In [1]:
import numpy as np
import argparse
import time
import cv2
import os
import glob
import imutils
import random as rng

In [2]:
net = cv2.dnn.readNet("models/v1/yolov3_8000.weights", "models/v1/yolov3.cfg")

In [3]:
LABELS = []
with open("models/v1/shopsign.names", "r") as f:
    LABELS = [line.strip() for line in f.readlines()]
#layer_names = net.getLayerNames()

In [4]:
#output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
image_list =  glob.glob('/home/visio-sevgi/Desktop/shopsign_project/shopsign_test_img/*jpg')

In [5]:
def rec_area(image):
    
    draw_list =[]
    Image = image.copy()
    
    #filters
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)

    medimg = cv2.medianBlur(gray,7)

    edged = cv2.Canny(medimg, 30, 200) #Canny(gray, 50, 100)
    #cv2.imshow("edged", edged)

    edged = cv2.dilate(edged, None, iterations=1)
    #cv2.imshow("Dilation", edged)


    edged = cv2.erode(edged, None, iterations=1)
    #cv2.imshow("Erode", edged)

    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    ori = image.copy()

    cnts = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) #RETR_EXTERNAL
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)
    #cv2.drawContours(image, cnts, -1, (255, 0, 0), 2)
    """
     cv2.imshow("All Contours", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    """
    x = []
    # find the ones with more than 200 area
    for c in cnts:
        if cv2.contourArea(c) > 10:
            arc_len = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.001 * arc_len, True)
            #print(len(approx))
            if (len(approx) > 3):
                #print("rectangle")
                x.append(approx)
    print(len(x))
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    #cv2.drawContours(ori, x, -1, color, 2)
    '''
    cv2.imshow("Contours", ori)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    '''
    for i in range(len(x)):
        xaxis = []
        yaxis = []
        #print("cokgenler")
        #print(x[i])
        for j in range(x[i].shape[0]):
            #print(i,j)
            #print(x[i][j])
            shape = np.reshape(x[i][j], (2,1))
            xaxis.append(int(shape[0]))
            yaxis.append(int(shape[1]))

        #print(xaxis,yaxis)
        top_left_x = min(xaxis)
        top_left_y = min(yaxis)
        bot_right_x = max(xaxis)
        bot_right_y = max(yaxis)
        rec = image[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]

        A = ((bot_right_y+1)-top_left_y)*((bot_right_x+1)-top_left_x)
        print("alan",A)
        if A > 10000 and A < int((image.shape[0]*image.shape[1])/2):
            draw_list.append((top_left_x,top_left_y,bot_right_x,bot_right_y))
        else:
            print("no")
    return draw_list
        
        
    

In [6]:
m = 0
#print(LABELS)
for i in image_list:
    image = cv2.imread(i)
    drawig_list = rec_area(image)
    #image = cv2.resize(image, None, fx=0.5, fy=0.5)
    image = cv2.resize(image,(1024,1024))
    (H, W) = image.shape[:2]
    print("image shape",H,W)
    blob = cv2.dnn.blobFromImage(image, 1 /300 , (H, W),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    load = time.time()
    # show timing information on YOLO
    print("YOLO took {:.6f} seconds".format(load - start))
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        my_traffic_signs = []
        # loop over each of the detections
        #print(len(output))
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detectiony
            scores = detection[5:]
            #print((scores))
            classID = np.argmax(scores)
            confidence = scores[classID]
            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > 0.0:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                #print(LABELS[classID])
                #print(confidence)
                #my_traffic_signs.append((LABELS[classID],confidence))
    #print(classIDs,confidences)
        end = time.time()
    print("1 frame :",end-start)
    #görselleştime
    idxs = cv2.dnn.NMSBoxes(boxes, confidences,0.1,0.5)
    #idxs2 = cv2.dnn.NMSBoxes(boxes, confidences,0.1,0.1)
    #print(classIDs)
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # draw a bounding box rectangle and label on the image
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
            #print(LABELS[classIDs[i]],confidence)
            print("...........>",text)
       
    else:
        print("no detect!",i)
 #show the output image
    for d in drawig_list:
        print(d)
        startp = (d[0],d[1])
        print("p1",startp)
        endp = (d[2], d[3])
        cv2.rectangle(image,startp ,endp , (0,255,0), 2)

    cv2.imshow("Image", image)
    cv2.waitKey(0)   
        
    print("=====================================================================================================")
    print("=====================================================================================================")

460
alan 252382
alan 1014013
no
alan 37200
alan 21670
alan 25872
alan 245565
alan 204918
alan 11036
alan 7107
no
alan 9212
no
alan 6708
no
alan 19575
alan 4365
no
alan 5754
no
alan 4444
no
alan 4699
no
alan 4018
no
alan 3492
no
alan 42872
alan 7332
no
alan 6390
no
alan 15606
alan 15750
alan 14790
alan 1971
no
alan 2047
no
alan 2050
no
alan 7171
no
alan 2067
no
alan 2835
no
alan 2322
no
alan 2860
no
alan 1722
no
alan 2832
no
alan 8576
no
alan 5341
no
alan 9500
no
alan 4848
no
alan 4836
no
alan 1482
no
alan 1482
no
alan 1891
no
alan 1340
no
alan 1443
no
alan 10530
alan 2461
no
alan 1036
no
alan 1072
no
alan 2380
no
alan 9152
no
alan 5896
no
alan 990
no
alan 1064
no
alan 3198
no
alan 1794
no
alan 1065
no
alan 1008
no
alan 83081
alan 1180
no
alan 1050
no
alan 820
no
alan 1242
no
alan 1496
no
alan 1121
no
alan 1850
no
alan 5633
no
alan 1872
no
alan 828
no
alan 779
no
alan 697
no
alan 792
no
alan 805
no
alan 900
no
alan 621
no
alan 2088
no
alan 1118
no
alan 2590
no
alan 1173
no
alan 682
no
a

YOLO took 2.047002 seconds
1 frame : 2.25059175491333
...........> tabela: 0.9223
...........> tabela: 0.6266
(985, 844, 1199, 1199)
p1 (985, 844)
(160, 0, 437, 209)
p1 (160, 0)
(1056, 934, 1181, 1132)
p1 (1056, 934)
(0, 448, 1199, 703)
p1 (0, 448)
(70, 588, 212, 662)
p1 (70, 588)
(0, 1099, 212, 1199)
p1 (0, 1099)
(0, 925, 473, 1134)
p1 (0, 925)
(165, 42, 436, 168)
p1 (165, 42)
(164, 230, 434, 564)
p1 (164, 230)
(499, 447, 1018, 532)
p1 (499, 447)
(435, 825, 1009, 1199)
p1 (435, 825)
(187, 112, 346, 179)
p1 (187, 112)
(64, 817, 259, 1021)
p1 (64, 817)
(166, 230, 435, 307)
p1 (166, 230)
(440, 627, 1112, 689)
p1 (440, 627)
(0, 666, 1199, 790)
p1 (0, 666)
(284, 926, 435, 1106)
p1 (284, 926)
(67, 0, 133, 243)
p1 (67, 0)
(279, 252, 407, 548)
p1 (279, 252)
(554, 225, 947, 325)
p1 (554, 225)
(563, 0, 946, 49)
p1 (563, 0)
656
alan 455282
alan 103222
alan 164775
alan 103113
alan 239375
alan 36955
alan 14110
alan 44394
alan 18972
alan 13419
alan 17114
alan 18792
alan 13920
alan 33276
alan 13419


YOLO took 2.072954 seconds
1 frame : 2.2723445892333984
...........> tabela: 0.8585
...........> tabela: 0.5109
...........> tabela: 0.2152
(196, 424, 1172, 889)
p1 (196, 424)
(230, 688, 740, 889)
p1 (230, 688)
(373, 0, 879, 324)
p1 (373, 0)
(1029, 0, 1199, 602)
p1 (1029, 0)
(272, 236, 896, 618)
p1 (272, 236)
(1034, 118, 1128, 506)
p1 (1034, 118)
(22, 418, 187, 502)
p1 (22, 418)
(1053, 798, 1199, 1099)
p1 (1053, 798)
(431, 627, 634, 719)
p1 (431, 627)
(516, 718, 704, 788)
p1 (516, 718)
(1074, 901, 1159, 1099)
p1 (1074, 901)
(853, 855, 933, 1086)
p1 (853, 855)
(853, 913, 932, 1086)
p1 (853, 913)
(0, 1012, 176, 1199)
p1 (0, 1012)
(516, 718, 704, 788)
p1 (516, 718)
(350, 723, 499, 799)
p1 (350, 723)
(350, 723, 499, 797)
p1 (350, 723)
(754, 123, 872, 246)
p1 (754, 123)
(95, 610, 413, 649)
p1 (95, 610)
(0, 672, 213, 880)
p1 (0, 672)
(689, 630, 749, 828)
p1 (689, 630)
(1112, 387, 1199, 655)
p1 (1112, 387)
(7, 0, 403, 132)
p1 (7, 0)
(30, 218, 198, 307)
p1 (30, 218)
(215, 651, 296, 776)
p1 (21

YOLO took 2.064189 seconds
1 frame : 2.2665796279907227
...........> tabela: 0.9552
...........> tabela: 0.9403
(216, 711, 1134, 1199)
p1 (216, 711)
(854, 310, 1128, 465)
p1 (854, 310)
(229, 335, 868, 436)
p1 (229, 335)
(218, 976, 686, 1199)
p1 (218, 976)
(887, 900, 1070, 1012)
p1 (887, 900)
(899, 1010, 1094, 1165)
p1 (899, 1010)
(343, 753, 610, 842)
p1 (343, 753)
(253, 352, 626, 408)
p1 (253, 352)
(854, 188, 1074, 267)
p1 (854, 188)
(892, 997, 1054, 1129)
p1 (892, 997)
(24, 747, 216, 858)
p1 (24, 747)
(860, 514, 1010, 586)
p1 (860, 514)
(394, 989, 684, 1077)
p1 (394, 989)
(237, 202, 553, 245)
p1 (237, 202)
(444, 29, 875, 88)
p1 (444, 29)
(89, 0, 1199, 317)
p1 (89, 0)
(0, 27, 283, 115)
p1 (0, 27)
(0, 356, 162, 466)
p1 (0, 356)
(719, 972, 816, 1078)
p1 (719, 972)
(0, 245, 169, 317)
p1 (0, 245)
(177, 1084, 604, 1199)
p1 (177, 1084)
(236, 911, 690, 984)
p1 (236, 911)
351
alan 271743
alan 37752
alan 40584
alan 40356
alan 33748
alan 80100
alan 47792
alan 8064
no
alan 6048
no
alan 7387
no
al

alan 2542
no
alan 128
no
alan 1827
no
alan 77
no
alan 126882
alan 88
no
alan 96
no
alan 70
no
alan 72
no
alan 96
no
alan 1725
no
alan 72
no
alan 70
no
alan 63
no
alan 66
no
alan 72
no
alan 63
no
alan 60
no
alan 56
no
alan 60
no
alan 63
no
alan 63
no
alan 64
no
alan 63
no
alan 56
no
alan 64
no
alan 60
no
alan 1947
no
alan 56
no
alan 64
no
alan 54
no
alan 56
no
alan 1720
no
alan 54
no
alan 54
no
alan 56
no
alan 64
no
alan 54
no
alan 50
no
alan 56
no
alan 54
no
alan 54
no
alan 56
no
alan 56
no
alan 48
no
alan 54
no
alan 48
no
alan 48
no
alan 56
no
alan 48
no
alan 48
no
alan 49
no
alan 48
no
alan 49
no
alan 49
no
alan 48
no
alan 49
no
alan 42
no
alan 42
no
alan 48
no
alan 42
no
alan 42
no
alan 49
no
alan 49
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 40
no
alan 40
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 42
no
alan 1575
no
alan 2379
no
alan 36
no
alan 567
no
alan 35
no
alan 35
no
alan 35
no
alan 35
no
alan 35
no
alan

...

alan 88
no
alan 110
no
alan 90
no
alan 99
no
alan 99
no
alan 90
no
alan 88
no
alan 90
no
alan 78
no
alan 90
no
alan 90
no
alan 90
no
alan 104
no
alan 88
no
alan 96
no
alan 90
no
alan 78
no
alan 456
no
alan 80
no
alan 90
no
alan 81
no
alan 84
no
alan 99
no
alan 81
no
alan 84
no
alan 1443
no
alan 72
no
alan 80
no
alan 72
no
alan 84
no
alan 91
no
alan 90
no
alan 80
no
alan 72
no
alan 72
no
alan 80
no
alan 72
no
alan 80
no
alan 70
no
alan 72
no
alan 66
no
alan 66
no
alan 77
no
alan 90
no
alan 72
no
alan 72
no
alan 72
no
alan 80
no
alan 72
no
alan 84
no
alan 81
no
alan 80
no
alan 72
no
alan 63
no
alan 63
no
alan 66
no
alan 81
no
alan 2880
no
alan 63
no
alan 80
no
alan 88
no
alan 64
no
alan 60
no
alan 60
no
alan 63
no
alan 63
no
alan 60
no
alan 63
no
alan 70
no
alan 64
no
alan 1802
no
alan 60
no
alan 64
no
alan 72
no
alan 63
no
alan 60
no
alan 56
no
alan 54
no
alan 64
no
alan 54
no
alan 56
no
alan 56
no
alan 56
no
alan 54
no
alan 50
no
alan 50
no
alan 50
no
alan 56
no
alan 56
no
alan 56
no
a

YOLO took 2.102888 seconds
1 frame : 2.311439275741577
no detect! /home/visio-sevgi/Desktop/shopsign_project/shopsign_test_img/2538.jpg
(404, 561, 796, 781)
p1 (404, 561)
(404, 561, 796, 645)
p1 (404, 561)
(191, 895, 905, 1142)
p1 (191, 895)
(1036, 537, 1199, 996)
p1 (1036, 537)
(805, 416, 1045, 546)
p1 (805, 416)
(47, 47, 200, 153)
p1 (47, 47)
(47, 47, 200, 147)
p1 (47, 47)
(472, 367, 564, 507)
p1 (472, 367)
(472, 369, 564, 506)
p1 (472, 369)
(388, 360, 464, 502)
p1 (388, 360)
(11, 503, 461, 531)
p1 (11, 503)
(782, 627, 864, 751)
p1 (782, 627)
(229, 1034, 725, 1135)
p1 (229, 1034)
(682, 897, 897, 985)
p1 (682, 897)
(856, 832, 971, 1056)
p1 (856, 832)
(1019, 0, 1199, 112)
p1 (1019, 0)
(0, 1022, 185, 1182)
p1 (0, 1022)
(27, 530, 300, 624)
p1 (27, 530)
(141, 1008, 219, 1143)
p1 (141, 1008)
(188, 1024, 672, 1135)
p1 (188, 1024)
(887, 1087, 1142, 1199)
p1 (887, 1087)
(188, 939, 583, 990)
p1 (188, 939)
512
alan 22326
alan 21112
alan 183534
alan 136485
alan 17316
alan 18150
alan 17331
alan 5

382
alan 343674
alan 65280
alan 9900
no
alan 112776
alan 4850
no
alan 4850
no
alan 20856
alan 3762
no
alan 60426
alan 83448
alan 4590
no
alan 13962
alan 3080
no
alan 2604
no
alan 3468
no
alan 2944
no
alan 3468
no
alan 2639
no
alan 4056
no
alan 105101
alan 2520
no
alan 2520
no
alan 5487
no
alan 12408
alan 54352
alan 1992
no
alan 1806
no
alan 2754
no
alan 4343
no
alan 2091
no
alan 3150
no
alan 2091
no
alan 2346
no
alan 2835
no
alan 8664
no
alan 8532
no
alan 1408
no
alan 1935
no
alan 1935
no
alan 1800
no
alan 5670
no
alan 1274
no
alan 1104
no
alan 1482
no
alan 1344
no
alan 1200
no
alan 1344
no
alan 1222
no
alan 92584
alan 1222
no
alan 1128
no
alan 1222
no
alan 1128
no
alan 1058
no
alan 1204
no
alan 1911
no
alan 860
no
alan 966
no
alan 10200
alan 880
no
alan 779
no
alan 738
no
alan 1140
no
alan 1008
no
alan 16027
alan 1008
no
alan 798
no
alan 2208
no
alan 4455
no
alan 1610
no
alan 1584
no
alan 676
no
alan 945
no
alan 714
no
alan 742
no
alan 4300
no
alan 884
no
alan 1218
no
alan 759
no
alan

YOLO took 2.078907 seconds
1 frame : 2.2780978679656982
...........> tabela: 0.8421
(0, 667, 270, 1060)
p1 (0, 667)
(301, 68, 1199, 506)
p1 (301, 68)
(350, 75, 727, 255)
p1 (350, 75)
(316, 302, 455, 507)
p1 (316, 302)
(271, 818, 623, 1045)
p1 (271, 818)
(17, 694, 270, 752)
p1 (17, 694)
(276, 849, 364, 984)
p1 (276, 849)
(915, 396, 1199, 496)
p1 (915, 396)
(349, 514, 890, 542)
p1 (349, 514)
(782, 348, 898, 527)
p1 (782, 348)
(298, 702, 417, 826)
p1 (298, 702)
(0, 363, 257, 551)
p1 (0, 363)
(0, 299, 247, 390)
p1 (0, 299)
(958, 874, 1063, 972)
p1 (958, 874)
(349, 0, 552, 185)
p1 (349, 0)
(729, 82, 920, 175)
p1 (729, 82)
(0, 759, 143, 851)
p1 (0, 759)
(0, 571, 309, 651)
p1 (0, 571)
445
alan 330528
alan 298596
alan 38130
alan 94856
alan 18681
alan 26163
alan 9900
no
alan 12089
alan 18060
alan 24332
alan 4850
no
alan 4850
no
alan 19000
alan 3724
no
alan 86268
alan 4590
no
alan 3080
no
alan 61336
alan 2511
no
alan 3468
no
alan 2944
no
alan 3468
no
alan 13328
alan 2275
no
alan 2584
no
alan 420

YOLO took 2.110365 seconds
1 frame : 2.3775975704193115
no detect! /home/visio-sevgi/Desktop/shopsign_project/shopsign_test_img/2558.jpg
(0, 702, 1063, 1175)
p1 (0, 702)
(57, 709, 329, 1101)
p1 (57, 709)
(277, 711, 352, 951)
p1 (277, 711)
(253, 905, 423, 1082)
p1 (253, 905)
(28, 0, 1077, 562)
p1 (28, 0)
(238, 1055, 442, 1161)
p1 (238, 1055)
(395, 121, 458, 301)
p1 (395, 121)
(462, 453, 600, 538)
p1 (462, 453)
(883, 18, 953, 169)
p1 (883, 18)
(977, 900, 1199, 1029)
p1 (977, 900)
(686, 533, 867, 637)
p1 (686, 533)
382
alan 337546
alan 179280
alan 19206
alan 15642
alan 42598
alan 50600
alan 6666
no
alan 42900
alan 8976
no
alan 8874
no
alan 44891
alan 4875
no
alan 14125
alan 8556
no
alan 3854
no
alan 2812
no
alan 7029
no
alan 2655
no
alan 3713
no
alan 116788
alan 2665
no
alan 2226
no
alan 6630
no
alan 22230
alan 12672
alan 5330
no
alan 2280
no
alan 1836
no
alan 1680
no
alan 1620
no
alan 2688
no
alan 3952
no
alan 6864
no
alan 5984
no
alan 11438
alan 1464
no
alan 1403
no
alan 1273
no
alan 95

In [7]:
drawig_list

[(705, 0, 1162, 736),
 (0, 319, 539, 650),
 (827, 342, 925, 535),
 (436, 494, 534, 651),
 (231, 647, 591, 764),
 (929, 650, 1148, 879),
 (598, 704, 817, 898),
 (229, 169, 599, 289),
 (841, 559, 953, 683),
 (0, 14, 678, 185),
 (570, 476, 664, 709),
 (840, 0, 938, 127),
 (674, 485, 759, 617),
 (1144, 578, 1199, 887),
 (0, 157, 666, 450),
 (474, 0, 585, 129),
 (300, 769, 626, 888),
 (815, 624, 934, 714),
 (768, 328, 917, 410),
 (0, 73, 137, 151),
 (448, 746, 574, 856),
 (79, 599, 181, 778)]

In [8]:
drawig_list[0][0]

705

In [9]:
startp

(79, 599)